In [2]:
import pandas as pd
import numpy as np

In [4]:

groupSize = 4
ntrials = 10
pi = 1.4
ntokens = 20
vals = np.arange(0,21,1) #possible values to contribute - from 0 to 20 tokens



rawDat = pd.read_csv("antisocialdataset.csv") # Public goods game

In [5]:
rawDat.head()

,sessionid,groupid,mgroupid,subjectid,p,secseq,city,period,senderscontribution,otherscontribution,...,civic,ruleoflaw,female,age,numknown,singlechild,ageu21,urbanbackground,middleclass,membership
0,011108KA,703,703,701,N-experiment,0,Samara,1,20,18,...,8.050.114,-0.884283,1,22.0,16.0,0.0,0,1.0,0.0,0.0
1,011108KA,703,703,701,N-experiment,0,Samara,1,20,10,...,8.050.114,-0.884283,1,22.0,16.0,0.0,0,1.0,0.0,0.0
2,011108KA,703,703,701,N-experiment,0,Samara,1,20,10,...,8.050.114,-0.884283,1,22.0,16.0,0.0,0,1.0,0.0,0.0
3,011108KA,702,702,702,N-experiment,0,Samara,1,10,10,...,8.050.114,-0.884283,1,21.0,10.0,0.0,0,1.0,1.0,1.0
4,011108KA,702,702,702,N-experiment,0,Samara,1,10,20,...,8.050.114,-0.884283,1,21.0,10.0,0.0,0,1.0,1.0,1.0


In [6]:
# create variable for GINI. Data from 
# http://hdr.undp.org/sites/default/files/reports/269/hdr_2009_en_complete.pdf,
rawDat.insert(22, 'gini', "") 

In [7]:
%%capture
rawDat['gini'][rawDat['city']=="Melbourne"]=34.3
rawDat['gini'][rawDat['city']=="Minsk"]=25.3
rawDat['gini'][rawDat['city']=="Chengdu"]=38.5
rawDat['gini'][rawDat['city']=="Copenhagen"]=28.7
rawDat['gini'][rawDat['city']=="Bonn"]=31.9
rawDat['gini'][rawDat['city']=="Athens"]=34.4
rawDat['gini'][rawDat['city']=="Seoul"]=31.6
rawDat['gini'][rawDat['city']=="Samara"]=37.5
rawDat['gini'][rawDat['city']=="Zurich"]=32.7
rawDat['gini'][rawDat['city']=="St. Gallen"]=32.7
rawDat['gini'][rawDat['city']=="Istanbul"]=41.9
rawDat['gini'][rawDat['city']=="Nottingham"]=34.8
rawDat['gini'][rawDat['city']=="Dnipropetrovs'k"]=26.1
rawDat['gini'][rawDat['city']=="Boston"]=41.1

In [8]:
# extract every third line - data file has lines representing others responses and we don't need that
redDat = rawDat.iloc[::3, :]

group_names = redDat.groupid.unique()
ngroups = len(group_names)


In [9]:
# THIS WILL REMOVE SUBJECTS WITH MISSING DATA IN NO PUNISHMENT CONDITION
ngroups = 269

subject_names = redDat['subjectid'].unique()
nsubjects = len(subject_names)


In [10]:
# data for no punishment condition #

c_no_punish = np.zeros(shape = (groupSize, ntrials, ngroups)) #groupSize = 4, ntrials = 10, groups = 269)
Ga_no_punish = np.zeros(shape = (ntrials, ngroups))   # shouldn't this be an average for each group? so groupSize instead of ntrials
Gc_no_punish = np.zeros(shape = (groupSize, ntrials, ngroups))
missing = np.zeros(ngroups)

In [11]:
for g in range(0,ngroups):
    c_no_punish[:,:,g][0] = np.array(redDat[(redDat['groupid']==group_names[g]) & (redDat['p']=="N-experiment")][['senderscontribution']][0:10]).flatten()
    c_no_punish[:,:,g][1] = np.array(redDat[(redDat['groupid']==group_names[g]) & (redDat['p']=="N-experiment")][['senderscontribution']][10:20]).flatten()
    c_no_punish[:,:,g][2] = np.array(redDat[(redDat['groupid']==group_names[g]) & (redDat['p']=="N-experiment")][['senderscontribution']][20:30]).flatten()
    c_no_punish[:,:,g][3] = np.array(redDat[(redDat['groupid']==group_names[g]) & (redDat['p']=="N-experiment")][['senderscontribution']][30:40]).flatten()
#take the average of the 10 trials for each group

    Ga_no_punish[:,g] = np.mean(c_no_punish[:,:,g], axis = 0)
    #missing[g] = np.isnan(c_no_punish[0][0][g])
    zero = np.delete(c_no_punish[:,:,:],[0], axis = 0)
    one = np.delete(c_no_punish[:,:,:],[1], axis = 0)
    two = np.delete(c_no_punish[:,:,:],[2], axis = 0)
    three = np.delete(c_no_punish[:,:,:],[3], axis = 0)

    Gc_no_punish[:,:,g][0] = np.sum(zero[:,:,g], axis = 0)
    Gc_no_punish[:,:,g][1] = np.sum(one[:,:,g], axis = 0)
    Gc_no_punish[:,:,g][2] = np.sum(two[:,:,g], axis = 0)
    Gc_no_punish[:,:,g][3] = np.sum(three[:,:,g], axis = 0)


In [12]:
c_no_punish

array([[[20., 10., 15., ...,  6., 10., 12.],
        [18., 10., 13., ..., 10.,  0., 15.],
        [10., 20.,  5., ..., 11., 10., 20.],
        ...,
        [20., 13., 15., ..., 18.,  0.,  8.],
        [ 5., 15., 20., ...,  4.,  1.,  1.],
        [15., 20., 10., ...,  0.,  5., 10.]],

       [[20., 20., 15., ..., 13.,  0., 20.],
        [15., 15., 10., ..., 20.,  0., 15.],
        [18., 18., 10., ...,  5.,  0.,  8.],
        ...,
        [19., 20., 20., ...,  5.,  0.,  5.],
        [12., 20.,  8., ..., 11.,  8., 15.],
        [13., 15., 15., ...,  0.,  0., 12.]],

       [[12.,  0., 10., ...,  1.,  1.,  0.],
        [20.,  5.,  9., ...,  2.,  0., 10.],
        [17., 20., 10., ...,  2.,  0.,  1.],
        ...,
        [ 3., 12., 14., ..., 20.,  0.,  5.],
        [ 3.,  1.,  5., ...,  4.,  0., 20.],
        [10., 10.,  8., ..., 14., 10.,  5.]],

       [[ 0., 20.,  1., ..., 14.,  0., 15.],
        [ 5., 15., 17., ..., 20.,  2., 10.],
        [ 0.,  8., 10., ..., 10.,  1., 17.],
        ..

In [14]:
# data for punishment condition #
c_punish = np.zeros(shape=( groupSize, ntrials, ngroups,))
Ga_punish = np.zeros(shape=(ntrials, ngroups))
Gc_punish = np.zeros(shape = ( groupSize, ntrials,ngroups))

In [15]:
for g in range(0,ngroups):
    c_punish[:,:,g][0] = np.array(redDat[(redDat['groupid']==group_names[g]) & (redDat['p']=="P-experiment")][['senderscontribution']][0:10]).flatten()
    c_punish[:,:,g][1] = np.array(redDat[(redDat['groupid']==group_names[g]) & (redDat['p']=="P-experiment")][['senderscontribution']][10:20]).flatten()
    c_punish[:,:,g][2] = np.array(redDat[(redDat['groupid']==group_names[g]) & (redDat['p']=="P-experiment")][['senderscontribution']][20:30]).flatten()
    c_punish[:,:,g][3] = np.array(redDat[(redDat['groupid']==group_names[g]) & (redDat['p']=="P-experiment")][['senderscontribution']][30:40]).flatten()
  
    Ga_punish[:,g] = np.mean(c_punish[:,:,g], axis = 0)
    
    zero_pun = np.delete(c_punish[:,:,:],[0], axis = 0)
    one_pun = np.delete(c_punish[:,:,:],[1], axis = 0)
    two_pun = np.delete(c_punish[:,:,:],[2], axis = 0)
    three_pun = np.delete(c_punish[:,:,:],[3], axis = 0)
    
    Gc_punish[:,:,g][0] = np.sum(zero_pun[:,:,g], axis = 0)
    Gc_punish[:,:,g][1] = np.sum(one_pun[:,:,g], axis = 0)
    Gc_punish[:,:,g][2] = np.sum(two_pun[:,:,g], axis = 0)
    Gc_punish[:,:,g][3] = np.sum(three_pun[:,:,g], axis = 0)
  


In [16]:
# compile data from each condition into 4D matrix

c = np.zeros(shape = (groupSize, ntrials, ngroups, 2))

c[:,:,:,0] = c_no_punish
c[:,:,:,1] = c_punish
Ga = np.zeros(shape = (ntrials, ngroups, 2))

In [20]:
c[:,:,:,1]

array([[[10., 15.,  1., ...,  4., 15., 17.],
        [14.,  5.,  0., ..., 10.,  0., 15.],
        [10., 10., 15., ..., 16., 10., 20.],
        ...,
        [13.,  0., 18., ..., 20.,  5., 19.],
        [ 5.,  0.,  2., ...,  2.,  8., 20.],
        [ 3.,  0., 20., ..., 20., 10., 20.]],

       [[10.,  0., 10., ..., 12., 10., 20.],
        [14.,  5., 15., ..., 20.,  7., 20.],
        [ 9.,  1.,  0., ...,  5., 12., 20.],
        ...,
        [ 2.,  3.,  2., ..., 10., 15., 20.],
        [10.,  0., 10., ..., 14., 13., 20.],
        [ 9.,  0.,  5., ..., 15., 12., 20.]],

       [[ 7.,  2., 20., ...,  4., 10., 20.],
        [ 9.,  0.,  1., ..., 13., 17., 20.],
        [10.,  1.,  0., ..., 14., 15., 20.],
        ...,
        [12.,  0., 15., ...,  0., 13., 20.],
        [12.,  5.,  0., ...,  3., 18., 20.],
        [15.,  2.,  0., ..., 13., 17., 20.]],

       [[15.,  0., 15., ...,  6., 18., 20.],
        [14.,  0., 15., ..., 15., 15., 20.],
        [20.,  0.,  1., ...,  4., 16., 20.],
        ..

In [21]:
Ga[:,:,0] = Ga_no_punish
Ga[:,:,1] = Ga_punish
Gc = np.zeros(shape = (groupSize, ntrials, ngroups, 2))
Gc[:,:,:,0]  = Gc_no_punish
Gc[:,:,:,1]  = Gc_punish
c_choice_index = c

In [25]:
Gini = np.zeros(shape = (ngroups))
for g in range(0, ngroups):
    try:
        Gini[g] = np.mean(redDat[(redDat['groupid'] == group_names[g]) & (redDat['p'] == 'P-experiment')][['gini']])
    except:
        Gini[g] = np.nan

In [26]:
nans = np.argwhere(np.isnan(Gini)).flatten()
Ga_punish = np.delete(Ga_punish, nans, axis = 1)
Ga_no_punish = np.delete(Ga_no_punish, nans, axis = 1)
c = np.delete(c, nans, axis = 2)
Ga = np.delete(Ga, nans, axis = 1)
Gc = np.delete(Gc, nans, axis = 2)
Gini = np.delete(Gini, nans, axis = 0)

In [27]:
#redefine number of groups after removing those without civic scores
ngroups = len(Gini)

In [28]:
nans

array([ 69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
       138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149])

In [ ]:
####SAVE EVERYTHING

In [119]:
np.save("c_no_punish.npy", c_no_punish)
np.save("Ga_no_punish.npy", Ga_no_punish)
np.save("Gc_no_punish.npy", Gc_no_punish)
np.save("c_punish.npy", c_punish)
np.save("Ga_punish.npy", Ga_punish)
np.save("Gc_punish.npy", Gc_punish)
np.save("c.npy", c)
np.save("Ga.npy", Ga)
np.save("Gc.npy", Gc)
np.save("c_choice_index.npy", c_choice_index)
np.save("Gini.npy", Gini)


In [29]:
c.shape

(4, 10, 219, 2)